In [2]:
# append the paths
import os

main_dir = os.getcwd().split("BERTMap")[0] + "BERTMap"
# os.environ["TOKENIZERS_PARALLELISM"] = "false"  # disable huggingface tokenizer paralellism
import sys

sys.path.append(main_dir)
import pandas as pd
from bertmap.onto import OntoBox, OntoEvaluator
from bertmap.extend import OntoExtend

In [3]:
na_vals = pd.io.parsers.STR_NA_VALUES.difference({"NULL", "null", "n/a"})
map_dir = main_dir + "/fma2snomed+.small/fine-tune.exp/us.ids.exp/map.200/extended/src.200.tsv"
map_file = OntoExtend.read_mappings_to_dict(map_dir, threshold=0.0)
src_onto = OntoBox.from_saved(main_dir + "/fma2snomed+.small/src.onto")
tgt_onto = OntoBox.from_saved(main_dir + "/fma2snomed+.small/tgt.onto")

read 10580 mappings with threshold >= 0.0.
found files of correct formats, trying to load ontology data from /home/yuahe/projects/BERTMap/fma2snomed+.small/src.onto
found files of correct formats, trying to load ontology data from /home/yuahe/projects/BERTMap/fma2snomed+.small/tgt.onto


In [4]:
lines = []
for m in map_file.keys():
    src_iri, tgt_iri = m.split("\t")
    src_iri = src_onto.onto_text.expand_entity_iri(src_iri)
    tgt_iri = tgt_onto.onto_text.expand_entity_iri(tgt_iri)
    value = map_file[m]
    lines.append(f"{src_iri}|{tgt_iri}|=|{value}|CLS\n")
    

In [5]:
with open(main_dir + "/fma2snomed+.small/fine-tune.exp/us.ids.exp/map.200/repaired/src.200.ex.txt", "w") as f:
    f.writelines(lines)

In [57]:
logmap_new = pd.read_csv(main_dir + "/testing/LogMap/output/mappings_repaired_with_LogMap.tsv", sep="\t", names=["Entity1", "Entity2", "Value"], na_values=na_vals, keep_default_na=False)
logmap_new["Entity1"] = logmap_new["Entity1"].apply(lambda iri: src_onto.onto_text.abbr_entity_iri(iri))
logmap_new["Entity2"] = logmap_new["Entity2"].apply(lambda iri: tgt_onto.onto_text.abbr_entity_iri(iri))

In [9]:
pred = main_dir + "/fma2snomed+.small/fine-tune.exp/us.ids.exp/map.200/repaired/src.200.tsv"
ref = main_dir + "/fma2snomed+.small/refs/maps.ref.full.tsv"
ignore = pd.read_csv(main_dir + "/fma2snomed+.small/refs/maps.ignored.tsv", sep="\t", na_values=na_vals, keep_default_na=False)
train_df = pd.read_csv(main_dir + "/fma2snomed+.small/refs/maps.ref.ss.train.tsv", sep="\t", na_values=na_vals, keep_default_na=False)
val_df = pd.read_csv(main_dir + "/fma2snomed+.small/refs/maps.ref.ss.val.tsv", sep="\t", na_values=na_vals, keep_default_na=False)
ignore = ignore.append(train_df).reset_index(drop=True)
ignore = ignore.append(val_df).reset_index(drop=True)

In [10]:
oe = OntoEvaluator(pred, ref, ignore)

FileNotFoundError: [Errno 2] No such file or directory: 'main_dir + /fma2snomed+.small/fine-tune.exp/us.ids.exp/map.200/repaired/src.200.tsv'

In [62]:
print(oe.P, oe.R, oe.F1)

0.9154551554462089 0.8489939846504875 0.8809728798966853
